# 10 Spectral Analysis

We begin with a trignometric function
$$x_t = A\cos(2\pi \omega t + \phi) = A\cos \phi \cos (2\pi \omega t) - A\sin\phi \sin(2\pi \omega t).$$

Parameter $\omega$ is the frequency, $A$ is the amplitude and  $\phi$ is the phase. Here $A$ and $\phi$ can be random variables. 

Let $U_1 = A\cos\phi$ while $U_2 = -A\sin\phi$, we can show that the following two assumptions are equivalent:
1. $U_1,U_2$ are independently sample from $N(0,1)$.
2. $A^2\sim \chi^2(2)$ and $\phi\sim U(-\pi,\pi)$ are independent.

## Periodogram

### Real DFT

Given $n$ observations $x_t$ where $n$ is odd, we can decompose by  
$$x_t = a_0+\sum_{j=1}^{(n-1)/2}\left(a_j\cos\frac{2\pi jt}{n}+b_j\sin \frac{2\pi jt}{n}\right)$$
where
$$a_j = \frac 2n\sum_{k=1}^n x_t \cos\frac{2\pi j t}{n}\quad {\rm and}\quad 
b_j = \frac 2n\sum_{k=1}^n x_t\sin \frac{2\pi jt}{n}.$$

The **scaled** periodogram is defined by
$$P\left(\frac jn \right) = a_j^2+b_j^2.$$

Large $P(j/n)$ implies predominance while small ones are overshadowed  by the noise.


### Complex DFT

Let $d(j/n)$ be defined as follows,
$$d\left(\frac jn\right) = \frac{1}{\sqrt n}\sum_{j=1}^n x_te^{-\frac{2\pi ijt}{n}}$$
where $i^2=-1$. We call its squared magnitude the periodogram, 
$$\left|d\left(\frac jn\right) \right|^2 = \frac 1n\left(\sum_{j=1}^n x_t\cos \frac{2\pi jt}{n}\right)^2
+\frac 1n \left(\sum_{j=1}^n x_t \sin \frac{2\pi jt}{n}\right)^2.$$
It has the relation that $P(j/n) = \frac 4n|d(j/n)|^2$.